In [1]:
!pip install streamlit



In [2]:
from flask import Flask, request, jsonify, render_template
import joblib
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import tensorflow as tf

# Load the trained models
log_reg_model = joblib.load('Logistic_Regression.joblib')
rf_model = joblib.load('Random_Forest.joblib')
gb_model = joblib.load('Gradient_Boosting.joblib')
nn_model = tf.keras.models.load_model('Neural_Network.h5')

# Load the dataset to fit the scaler (assuming the dataset file is available)
file_path = 'heart_failure_clinical_records_dataset.csv'
data = pd.read_csv(file_path)
X = data.drop(columns=['DEATH_EVENT'])

# Create a Flask app
app = Flask(__name__)

# Define the home route
@app.route('/')
def home():
    return render_template('index.html')

# Define the prediction route
@app.route('/predict', methods=['POST'])
def predict():
    # Get user input from the form
    model_choice = request.form['model']
    age = float(request.form['age'])
    anaemia = int(request.form['anaemia'])
    creatinine_phosphokinase = float(request.form['creatinine_phosphokinase'])
    diabetes = int(request.form['diabetes'])
    ejection_fraction = float(request.form['ejection_fraction'])
    high_blood_pressure = int(request.form['high_blood_pressure'])
    platelets = float(request.form['platelets'])
    serum_creatinine = float(request.form['serum_creatinine'])
    serum_sodium = float(request.form['serum_sodium'])
    sex = int(request.form['sex'])
    smoking = int(request.form['smoking'])
    time = float(request.form['time'])

    # Create an array of the input values
    input_data = np.array([[age, anaemia, creatinine_phosphokinase, diabetes, ejection_fraction,
                            high_blood_pressure, platelets, serum_creatinine, serum_sodium, sex,
                            smoking, time]])

    # Scale the input data
    scaler = StandardScaler()
    scaler.fit(X)
    input_data_scaled = scaler.transform(input_data)

    # Predict using the selected model
    if model_choice == 'Logistic Regression':
        pred = log_reg_model.predict(input_data_scaled)
    elif model_choice == 'Random Forest':
        pred = rf_model.predict(input_data_scaled)
    elif model_choice == 'Gradient Boosting':
        pred = gb_model.predict(input_data_scaled)
    elif model_choice == 'Neural Network':
        pred = (nn_model.predict(input_data_scaled) > 0.5).astype("int32")

    result = int(pred[0]) if model_choice != 'Neural Network' else int(pred[0][0])

    return jsonify({'prediction': result})

if __name__ == '__main__':
    app.run(debug=True)


C:\Users\potab\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\potab\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (windowsapi)


SystemExit: 1

C:\Users\potab\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
<!DOCTYPE html>
<html>
<head>
    <title>Readmission Prediction</title>
</head>
<body>
    <h1>Predict Hospital Readmission</h1>
    <form id="prediction-form">
        <label>Age:</label>
        <input type="number" name="age" step="any" required><br>
        <label>Anaemia (0 or 1):</label>
        <input type="number" name="anaemia" min="0" max="1" required><br>
        <label>Creatinine Phosphokinase:</label>
        <input type="number" name="creatinine_phosphokinase" step="any" required><br>
        <label>Diabetes (0 or 1):</label>
        <input type="number" name="diabetes" min="0" max="1" required><br>
        <label>Ejection Fraction:</label>
        <input type="number" name="ejection_fraction" step="any" required><br>
        <label>High Blood Pressure (0 or 1):</label>
        <input type="number" name="high_blood_pressure" min="0" max="1" required><br>
        <label>Platelets:</label>
        <input type="number" name="platelets" step="any" required><br>
        <label>Serum Creatinine:</label>
        <input type="number" name="serum_creatinine" step="any" required><br>
        <label>Serum Sodium:</label>
        <input type="number" name="serum_sodium" step="any" required><br>
        <label>Sex (0 or 1):</label>
        <input type="number" name="sex" min="0" max="1" required><br>
        <label>Smoking (0 or 1):</label>
        <input type="number" name="smoking" min="0" max="1" required><br>
        <label>Time:</label>
        <input type="number" name="time" step="any" required><br>
        <label>Select Model:</label>
        <select name="model" required>
            <option value="Logistic Regression">Logistic Regression</option>
            <option value="Random Forest">Random Forest</option>
            <option value="Gradient Boosting">Gradient Boosting</option>
            <option value="Neural Network">Neural Network</option>
        </select><br>
        <input type="submit" value="Predict">
    </form>
    <div id="results"></div>
</body>
<script>
    document.getElementById('prediction-form').onsubmit = async function(event) {
        event.preventDefault();
        const formData = new FormData(this);
        const response = await fetch('/predict', {
            method: 'POST',
            body: formData
        });
        const result = await response.json();
        document.getElementById('results').innerText = `Prediction: ${result.prediction}`;
    };
</script>
</html>


In [ ]:
joblib.dump(scaler, 'scaler.joblib')
joblib.dump(log_reg_model, 'Logistic_Regression.joblib')
joblib.dump(rf_model, 'Random_Forest.joblib')
joblib.dump(gb_model, 'Gradient_Boosting.joblib')
nn_model.save('Neural_Network.h5')


In [ ]:
python app.py
